In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler

In [2]:
# load regional structural measures
CT_data = pd.read_csv('raw_data/ukbb_CT_DK.csv').dropna(how='any')
CSA_data = pd.read_csv('raw_data/ukbb_CSA_DK.csv').dropna(how='any')
sub_data = pd.read_csv('raw_data/ukbb_sub_volumes.csv').dropna(how='any')

In [3]:
# load global structural measures
global_data = pd.read_csv('raw_data/global_brain_measures.csv').dropna(how='any')

# load scanning positions x,y,z
pos_data = pd.read_csv('raw_data/scanning_positions.csv').dropna(how='any')

# load covariates 
age_sex = pd.read_csv('raw_data/ukbb_age_sex.csv').dropna(how='any')
cols = ['eid']
for i in range(25):
    cols.append('PC'+str(i+1))
PCs = pd.read_csv('raw_data/top_100_PCs.csv')[cols].dropna(how='any')

In [4]:
# load PRS data
PRS_data = pd.read_csv('raw_data/multi_PRSs.csv').dropna(how='any')

In [5]:
# merge data
CT_data.set_index('eid',inplace=True)
CSA_data.set_index('eid',inplace=True)
sub_data.set_index('eid',inplace=True)
global_data.set_index('eid',inplace=True)
pos_data.set_index('eid',inplace=True)
age_sex.set_index('eid',inplace=True)
PCs.set_index('eid',inplace=True)
PRS_data.set_index('eid',inplace=True)

l = list(set(CT_data.index) & set(CSA_data.index) & set(sub_data.index) \
         & set(global_data.index) & set(pos_data.index) & set(age_sex.index) & set(PCs.index) & set(PRS_data.index))

final_CT = CT_data.loc[l]
final_CSA = CSA_data.loc[l]
final_sub = sub_data.loc[l]
final_global_measures = global_data.loc[l]
final_pos = pos_data.loc[l]
final_age_sex = age_sex.loc[l]
final_PCs = PCs.loc[l]
final_PRSs = PRS_data.loc[l]

final_CT.reset_index(inplace=True)
final_CSA.reset_index(inplace=True)
final_sub.reset_index(inplace=True)
final_global_measures.reset_index(inplace=True)
final_pos.reset_index(inplace=True)
final_age_sex.reset_index(inplace=True)
final_PCs.reset_index(inplace=True)
final_PRSs.reset_index(inplace=True)

In [6]:
# create covariates
ACT = final_global_measures.iloc[:,3:4].values
TCSA = final_global_measures.iloc[:,2:3].values
ICV = final_global_measures.iloc[:,1:2].values

PCs_25 = final_PCs.iloc[:,1:].values
age = final_age_sex.iloc[:,2:3].values
sex = final_age_sex.iloc[:,1:2].values
postions = final_pos.iloc[:,1:].values
sex = sex + 1

co_for_CT = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,ACT))
co_for_CSA = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,TCSA))
co_for_sub = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions,ICV))
co = np.hstack((PCs_25,age,sex,age*age,age*sex,age*age*sex,postions))

In [7]:
# function used for regressing out the effects of covariates 
from sklearn.preprocessing import StandardScaler

def regression_covariant(covariant_matrix, y, standard_scale=False):
    a = np.hstack((covariant_matrix,np.ones((covariant_matrix.shape[0], 1))))
    w = np.linalg.lstsq(a,y,rcond=None)[0]

    residual = y - covariant_matrix.dot(w[:-1])
    residual = residual.astype('float64')

    if standard_scale:
        residual = StandardScaler().fit_transform(residual.reshape(-1,1)).flatten()

    return residual, w

In [8]:
X = final_PRSs.iloc[:,1:].values
X[:,7] = -X[:,7]
X[:,8] = -X[:,8]
X[:,11] = -X[:,11]

Y_CT = final_CT.iloc[:,1:].values
Y_CSA = final_CSA.iloc[:,1:].values
Y_sub = final_sub.iloc[:,1:].values

In [10]:
from scipy import stats

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [11]:
def correlation_analysis(X,Y,co):
    re_R = np.empty((X.shape[1],Y.shape[1]))
    re_P = np.empty((X.shape[1],Y.shape[1]))
    Low = np.empty((X.shape[1],Y.shape[1]))
    High = np.empty((X.shape[1],Y.shape[1]))
    for i in range(X.shape[1]):
        for j in range(Y.shape[1]):
            x = X[:,i]
            y = Y[:,j]
            [rx,w1] = regression_covariant(co,x,standard_scale=True)
            [ry,w1] = regression_covariant(co,y,standard_scale=True)
            r,p,lo,hi = pearsonr_ci(rx, ry)
            re_R[i,j] = r
            re_P[i,j] = p
            Low[i,j] = lo
            High[i,j] = hi
    return re_R,re_P,Low,High

In [13]:
re_CT_R, re_CT_P, Low_CT, High_CT= correlation_analysis(X,Y_CT,co_for_CT)
re_CSA_R, re_CSA_P, Low_CSA, High_CSA = correlation_analysis(X,Y_CSA,co_for_CSA)
re_sub_R, re_sub_P, Low_sub, High_sub = correlation_analysis(X,Y_sub,co_for_sub)

In [28]:
def output_CIs(Low,High,output_path):
    s = Low.shape
    for i in range(s[0]):
        tmp = []
        for j in range(s[1]):
            lo = round(Low[i,j], 4)
            hi = round(High[i,j], 4)
            tmp.append('(' + str(lo) + ' , ' + str(hi) + ')')
        if i == 0:
            data = pd.DataFrame(data=tmp,columns=[str(i+1)])
        else:
            data[str(i+1)] = tmp
    output_file = output_path + '_CIs.csv'
    data.to_csv(output_file,index=False)

In [29]:
output_CIs(Low_CT, High_CT,'results/regional_structural_measures/CT')
output_CIs(Low_CSA, High_CSA,'results/regional_structural_measures/CSA')
output_CIs(Low_sub, High_sub,'results/regional_structural_measures/volume_sub')

In [38]:
# merge their results (P values) together
total_P = np.hstack((re_CT_P,re_CSA_P,re_sub_P))

In [39]:
from statsmodels.stats import multitest

def fdr_correction(P):
    size = P.shape
    temp_p = P.flatten()
    Ps = multitest.multipletests(temp_p,alpha=0.05,method='fdr_bh')
    P_corrected = Ps[1].reshape(size)
    return P_corrected

In [40]:
correct_P = fdr_correction(total_P)

In [41]:
region_num = 33
re_CT_P_corrected = correct_P[:,:2*region_num]
re_CSA_P_corrected = correct_P[:,2*region_num:4*region_num]
re_sub_P_corrected = correct_P[:,4*region_num:]

In [42]:
# output CSA results
lSA_R = re_CSA_R[:,:region_num]
lSA_P = re_CSA_P[:,:region_num]
lSA_P_corrected = re_CSA_P_corrected[:,:region_num]
re_l_R = pd.DataFrame(data=lSA_R)
re_l_R.to_csv('results/regional_structural_measures/CSA_left_R.csv',header=None,index=False)
re_l_P = pd.DataFrame(data=lSA_P)
re_l_P.to_csv('results/regional_structural_measures/CSA_left_P.csv',header=None,index=False)
re_l_P_corrected = pd.DataFrame(data=lSA_P_corrected)
re_l_P_corrected.to_csv('results/regional_structural_measures/CSA_left_P_corrected.csv',header=None,index=False)

rSA_R = re_CSA_R[:,region_num:]
rSA_P = re_CSA_P[:,region_num:]
rSA_P_corrected = re_CSA_P_corrected[:,region_num:]
re_r_R = pd.DataFrame(data=rSA_R)
re_r_R.to_csv('results/regional_structural_measures/CSA_right_R.csv',header=None,index=False)
re_r_P = pd.DataFrame(data=rSA_P)
re_r_P.to_csv('results/regional_structural_measures/CSA_right_P.csv',header=None,index=False)
re_r_P_corrected = pd.DataFrame(data=rSA_P_corrected)
re_r_P_corrected.to_csv('results/regional_structural_measures/CSA_right_P_corrected.csv',header=None,index=False)

In [43]:
# output CT results
lCT_R = re_CT_R[:,:region_num]
lCT_P = re_CT_P[:,:region_num]
lCT_P_corrected = re_CT_P_corrected[:,:region_num]
re_l_R = pd.DataFrame(data=lCT_R)
re_l_R.to_csv('results/regional_structural_measures/CT_left_R.csv',header=None,index=False)
re_l_P = pd.DataFrame(data=lCT_P)
re_l_P.to_csv('results/regional_structural_measures/CT_left_P.csv',header=None,index=False)
re_l_P_corrected = pd.DataFrame(data=lCT_P_corrected)
re_l_P_corrected.to_csv('results/regional_structural_measures/CT_left_P_corrected.csv',\
                        header=None,index=False)

rCT_R = re_CT_R[:,region_num:]
rCT_P = re_CT_P[:,region_num:]
rCT_P_corrected = re_CT_P_corrected[:,region_num:]
re_r_R = pd.DataFrame(data=rCT_R)
re_r_R.to_csv('results/regional_structural_measures/CT_right_R.csv',header=None,index=False)
re_r_P = pd.DataFrame(data=rCT_P)
re_r_P.to_csv('results/regional_structural_measures/CT_right_P.csv',header=None,index=False)
re_r_P_corrected = pd.DataFrame(data=rCT_P_corrected)
re_r_P_corrected.to_csv('results/regional_structural_measures/CT_right_P_corrected.csv',\
                        header=None,index=False)

In [44]:
# output results of subcortical volumes 
subvol_R = re_sub_R
subvol_P = re_sub_P
subvol_P_corrected = re_sub_P_corrected
re_sub_R = pd.DataFrame(data=subvol_R)
re_sub_R.to_csv('results/regional_structural_measures/volume_sub_R.csv',header=None,index=False)
re_sub_P = pd.DataFrame(data=subvol_P)
re_sub_P.to_csv('results/regional_structural_measures/volume_sub_P.csv',header=None,index=False)
re_sub_P_corrected = pd.DataFrame(data=subvol_P_corrected)
re_sub_P_corrected.to_csv('results/regional_structural_measures/volume_sub_P_corrected.csv',header=None,index=False)